In [ ]:
import pyvista as pv
import numpy as np

# Read the data
mesh = pv.read(r"C:\Users\KU5001153\Downloads\bp_50percent_75.vtk")

# Create a slice
slice_x = mesh.slice(normal='z', origin=(0, 0, 50))

# Clip the slice to show only z >= 60
clipped_slice = slice_x.clip(normal=[0, 0, -1], origin=[0, 0, 0])

# Create glyphs with adjusted parameters for better visualization
glyphs = clipped_slice.glyph(
    orient='U',              
    scale='U',              
    factor=0.2,             
    geom=pv.Arrow()         
)

# Create the plotter
plotter = pv.Plotter()

# Add the slice plane with enhanced labeling
plotter.add_mesh(clipped_slice, 
                opacity=1.0,
                cmap='inferno',
                clim=[0, 11],
                scalar_bar_args={
                    'title': 'Velocity Magnitude (m/s)',
                    'n_labels': 6,
                    'position_x': 0.20,
                    'position_y': 0.75
                })

# Add axes with labels
plotter.add_axes(
    xlabel='X (mm)',
    ylabel='Y (mm)',
    zlabel='Z (mm)',
    line_width=3,
    viewport=(0, 0, 0.2, 0.2),
)



# Add bounds with reduced font size
plotter.show_bounds(
    show_xaxis=True,
    show_yaxis=True,
    show_zaxis=True,
    font_size=13,  # Reduced font size
    font_family='arial',
    show_xlabels=True,
    show_ylabels=True,
    show_zlabels=True,
)

# Set view to be normal to z-axis
# plotter.view_xy()  # This sets view normal to z-axis
plotter.camera.zoom(1.1)
# Show the plot
plotter.show()

In [ ]:
import pyvista as pv
import numpy as np

# Read the data
mesh = pv.read(r"C:\Users\KU5001153\Downloads\bp_50percent_75.vtk")
# Get bounds
bounds = mesh.bounds
z_min, z_max = bounds[4], bounds[5]

# Create plotter with static background
plotter = pv.Plotter(lighting='none')

# Create slices
z_positions = np.linspace(z_min, z_max, 30)
for z_pos in z_positions:
    slice_z = mesh.slice(normal='z', origin=(0, 0, z_pos))
    if slice_z.n_points > 0:
        plotter.add_mesh(slice_z, 
                        opacity=0.1,
                        cmap='inferno',
                        clim=[0, 13],
                        scalars='U',
                        scalar_bar_args={
                            'title': 'Velocity Magnitude (m/s)',
                            'n_labels': 6,
                            'position_x': 0.20,
                            'position_y': 0.82
                        },
                        lighting=False,
                        show_edges=False,
                        interpolate_before_map=False)

# Add simple axes with just X, Y, Z labels
plotter.add_axes(
    xlabel='X',
    ylabel='Y',
    zlabel='Z',
    line_width=2,
)

# Increase window size
plotter.window_size = [1200, 800]
plotter.camera.zoom(1.1)
plotter.show()

In [ ]:
import pyvista as pv
import numpy as np
from scipy.interpolate import griddata

# Read and prepare data
mesh = pv.read(r"C:\Users\KU5001153\Downloads\bp_50percent_75.vtk")
slice_x = mesh.slice(normal='z', origin=(0, 0, 60))
clipped_slice = slice_x.clip(normal=[0, 0, -1], origin=[0, 0, 0])

# Get points and velocity
points = clipped_slice.points
velocity = clipped_slice['U']

# Create regular grid for interpolation
x = points[:, 0]
y = points[:, 1]
xi = np.linspace(x.min(), x.max(), 100)
yi = np.linspace(y.min(), y.max(), 100)
xi, yi = np.meshgrid(xi, yi)

# Interpolate velocity components
u = griddata((x, y), velocity[:, 0], (xi, yi), method='cubic')
v = griddata((x, y), velocity[:, 1], (xi, yi), method='cubic')
w = griddata((x, y), velocity[:, 2], (xi, yi), method='cubic')

# Calculate gradients
dx = (x.max() - x.min()) / (xi.shape[1] - 1)
dy = (y.max() - y.min()) / (xi.shape[0] - 1)

dudx, dudy = np.gradient(u, dx, dy)
dvdx, dvdy = np.gradient(v, dx, dy)
dwdx, dwdy = np.gradient(w, dx, dy)

# Interpolate gradients back to original points
dudx_orig = griddata((xi.flatten(), yi.flatten()), dudx.flatten(), (x, y), method='linear')
dudy_orig = griddata((xi.flatten(), yi.flatten()), dudy.flatten(), (x, y), method='linear')
dvdx_orig = griddata((xi.flatten(), yi.flatten()), dvdx.flatten(), (x, y), method='linear')
dvdy_orig = griddata((xi.flatten(), yi.flatten()), dvdy.flatten(), (x, y), method='linear')
dwdx_orig = griddata((xi.flatten(), yi.flatten()), dwdx.flatten(), (x, y), method='linear')
dwdy_orig = griddata((xi.flatten(), yi.flatten()), dwdy.flatten(), (x, y), method='linear')

# Add to dataset
clipped_slice['dudx'] = dudx_orig
clipped_slice['dudy'] = dudy_orig
clipped_slice['dvdx'] = dvdx_orig
clipped_slice['dvdy'] = dvdy_orig
clipped_slice['dwdx'] = dwdx_orig
clipped_slice['dwdy'] = dwdy_orig

# Plot
plotter = pv.Plotter()
plotter.add_mesh(clipped_slice, 
                scalars='dudx',
                opacity=1.0,
                cmap='viridis',
                scalar_bar_args={'title': 'du/dx'})

plotter.add_axes()
plotter.camera.elevation = 0
plotter.camera.azimuth = 0
plotter.show()

In [ ]:
import pyvista as pv
import numpy as np

# Read the data
mesh = pv.read(r"C:\Users\KU5001153\Downloads\bp_50percent_75.vtk")
# Get bounds
bounds = mesh.bounds
z_min, z_max = bounds[4], bounds[5]

# Create plotter with static background
plotter = pv.Plotter(lighting='none')

# Create slices
z_positions = np.linspace(z_min, z_max, 30)
for z_pos in z_positions:
    slice_z = mesh.slice(normal='z', origin=(0, 0, z_pos))
    if slice_z.n_points > 0:
        plotter.add_mesh(slice_z, 
                        opacity=0.1,
                        cmap='inferno',
                        clim=[0, 13],
                        scalars='U',
                        scalar_bar_args={
                            'title': 'Velocity Magnitude (m/s)',
                            'n_labels': 6,
                            'position_x': 0.20,
                            'position_y': 0.82
                        },
                        lighting=False,
                        show_edges=False,
                        interpolate_before_map=False)

# Add simple axes with just X, Y, Z labels
plotter.add_axes(
    xlabel='X',
    ylabel='Y',
    zlabel='Z',
    line_width=2,
)

# Increase window size
plotter.window_size = [1200, 800]
plotter.camera.zoom(1.1)
plotter.show()

In [ ]:
import pyvista as pv
import numpy as np

# Read the data
mesh = pv.read(r"C:\Users\KU5001153\Downloads\bp_50percent_75.vtk")
# Get bounds
bounds = mesh.bounds
z_min, z_max = bounds[4], bounds[5]

# Create plotter with static background
plotter = pv.Plotter(lighting='none')

# Create slices
z_positions = np.linspace(z_min, z_max, 30)
for z_pos in z_positions:
    slice_z = mesh.slice(normal='z', origin=(0, 0, z_pos))
    if slice_z.n_points > 0:
        plotter.add_mesh(slice_z, 
                        opacity=0.1,
                        cmap='inferno',
                        clim=[0, 13],
                        scalars='U',
                        scalar_bar_args={
                            'title': 'Velocity Magnitude (m/s)',
                            'n_labels': 6,
                            'position_x': 0.20,
                            'position_y': 0.82
                        },
                        lighting=False,
                        show_edges=False,
                        interpolate_before_map=False)

# Add simple axes with just X, Y, Z labels
plotter.add_axes(
    xlabel='X',
    ylabel='Y',
    zlabel='Z',
    line_width=2,
)

# Increase window size
plotter.window_size = [1200, 800]
plotter.camera.zoom(1.1)
plotter.show()

In [ ]:
import pyvista as pv
import numpy as np

mesh = pv.read(r"C:\Users\KU5001153\Downloads\bp_50percent_147.vtk")
x_positions = np.linspace(100, 500, 15)
plotter = pv.Plotter(shape=(2, 2))

# Setup subplots
plotter.subplot(0, 0)
plotter.add_text("Velocity Magnitude", font_size=10)
plotter.subplot(0, 1)
plotter.add_text("du/dx", font_size=10)
plotter.subplot(1, 0)
plotter.add_text("dv/dx", font_size=10)
plotter.subplot(1, 1)
plotter.add_text("dw/dx", font_size=10)

spacing = 20  # Space between slices

# First do velocity magnitude separately
plotter.subplot(0, 0)
for i, x_pos in enumerate(x_positions):
    slice_x = mesh.slice(normal='x', origin=(x_pos, 0, 0))
    clipped_slice = slice_x.clip(normal=[-1, 0, 0], origin=[0, 0, 0])
    clipped_slice = clipped_slice.clip(normal=[0, 0, -1], origin=[0, 0, 0])
    clipped_slice = clipped_slice.clip(normal=[0, -1, 0], origin=[0, 70, 0])
    clipped_slice.translate((0, i*spacing, 0))
    u = clipped_slice.get_array('U')
    velocity_magnitude = np.linalg.norm(u, axis=1)
    clipped_slice['velocity_magnitude'] = velocity_magnitude
    plotter.add_mesh(clipped_slice,
                    scalars='velocity_magnitude',
                    opacity=0.8,
                    cmap='rainbow',
                    clim=[0, 10],
                    scalar_bar_args={'title': 'Velocity Magnitude (m/s)'})

# Then do gradients in other subplots
for i, x_pos in enumerate(x_positions):
    if i > 0:
        # Current slice
        slice_x = mesh.slice(normal='x', origin=(x_pos, 0, 0))
        clipped_slice = slice_x.clip(normal=[-1, 0, 0], origin=[0, 0, 0])
        clipped_slice = clipped_slice.clip(normal=[0, 0, -1], origin=[0, 0, 0])
        clipped_slice = clipped_slice.clip(normal=[0, -1, 0], origin=[0, 70, 0])
        clipped_slice.translate((0, i*spacing, 0))
        u = clipped_slice.get_array('U')
        
        # Previous slice with same clipping
        prev_slice = mesh.slice(normal='x', origin=(x_positions[i-1], 0, 0))
        prev_slice = prev_slice.clip(normal=[-1, 0, 0], origin=[0, 0, 0])
        prev_slice = prev_slice.clip(normal=[0, 0, -1], origin=[0, 0, 0])
        prev_slice = prev_slice.clip(normal=[0, -1, 0], origin=[0, 70, 0])
        prev_u = prev_slice.get_array('U')
        
        # Calculate gradients
        dx = x_pos - x_positions[i-1]
        du_dx = (u[:, 0] - prev_u[:, 0]) / dx
        dv_dx = (u[:, 1] - prev_u[:, 1]) / dx
        dw_dx = (u[:, 2] - prev_u[:, 2]) / dx
        
        clipped_slice['du_dx'] = du_dx
        clipped_slice['dv_dx'] = dv_dx
        clipped_slice['dw_dx'] = dw_dx
        
        # Add to gradient subplots
        plotter.subplot(0, 1)
        plotter.add_mesh(clipped_slice,
                        scalars='du_dx',
                        opacity=0.8,
                        cmap='rainbow',
                        clim=[-0.1, 0.1],
                        scalar_bar_args={'title': 'du/dx (1/s)'})
        
        plotter.subplot(1, 0)
        plotter.add_mesh(clipped_slice,
                        scalars='dv_dx',
                        opacity=0.8,
                        cmap='rainbow',
                        clim=[-0.1, 0.1],
                        scalar_bar_args={'title': 'dv/dx (1/s)'})
        
        plotter.subplot(1, 1)
        plotter.add_mesh(clipped_slice,
                        scalars='dw_dx',
                        opacity=0.8,
                        cmap='rainbow',
                        clim=[-0.1, 0.1],
                        scalar_bar_args={'title': 'dw/dx (1/s)'})

# Add axes to all subplots
for i in range(2):
    for j in range(2):
        plotter.subplot(i, j)
        plotter.add_axes()

plotter.window_size = [1600, 1200]
plotter.camera.zoom(1.1)
plotter.show()

In [ ]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

mesh = pv.read(r"C:\Users\KU5001153\Downloads\bp_50percent_75.vtk")
x_positions = np.linspace(100, 500, 15)

# Arrays to store values at y=100, z=50
u_values = []
v_values = []
w_values = []
du_dx_values = []
dv_dx_values = []
dw_dx_values = []

for i, x_pos in enumerate(x_positions):
    
    slice_x = mesh.slice(normal='x', origin=(x_pos, 0, 0))
    clipped_slice = slice_x.clip(normal=[-1, 0, 0], origin=[0, 0, 0])
    clipped_slice = clipped_slice.clip(normal=[0, 0, -1], origin=[0, 0, 0])
    clipped_slice = clipped_slice.clip(normal=[0, -1, 0], origin=[0, 70, 0])
    
    
    points = clipped_slice.points
    u = clipped_slice.get_array('U')
    
    distances = np.sqrt((points[:, 1] - 100)**2 + (points[:, 2] - 50)**2)
    closest_point_idx = np.argmin(distances)
    
    u_values.append(u[closest_point_idx, 0])
    v_values.append(u[closest_point_idx, 1])
    w_values.append(u[closest_point_idx, 2])
    
    if i > 0:
        dx = x_pos - x_positions[i-1]
        du_dx_values.append((u_values[-1] - u_values[-2]) / dx)
        dv_dx_values.append((v_values[-1] - v_values[-2]) / dx)
        dw_dx_values.append((w_values[-1] - w_values[-2]) / dx)

fig = plt.figure(figsize=(12, 8))
gs = GridSpec(2, 1, height_ratios=[1, 1], hspace=0.3)

colors = ['#0072BD', '#D95319', '#77AC30']  # Blue, Orange, Green

# Plot velocity components
ax1 = fig.add_subplot(gs[0])
ax1.plot(x_positions, u_values, color=colors[0], label='u', linewidth=2, marker='o')
ax1.plot(x_positions, v_values, color=colors[1], label='v', linewidth=2, marker='s')
ax1.plot(x_positions, w_values, color=colors[2], label='w', linewidth=2, marker='^')
ax1.set_xlabel('x position (m)', fontsize=12)
ax1.set_ylabel('Velocity (m/s)', fontsize=12)
ax1.set_title('Wind Velocity Components at y=100m, z=50m', fontsize=14, pad=15)
ax1.legend(fontsize=10, loc='best')
ax1.grid(False)
ax1.tick_params(axis='both', labelsize=10)

ax2 = fig.add_subplot(gs[1])
ax2.plot(x_grad, du_dx_values, color=colors[0], label='du/dx', linewidth=2, marker='o')
ax2.plot(x_grad, dv_dx_values, color=colors[1], label='dv/dx', linewidth=2, marker='s')
ax2.plot(x_grad, dw_dx_values, color=colors[2], label='dw/dx', linewidth=2, marker='^')
ax2.set_xlabel('x position (m)', fontsize=12)
ax2.set_ylabel('Wind Velocity Gradient (1/s)', fontsize=12)
ax2.set_title('Velocity Gradients at y=100m, z=50m', fontsize=14, pad=15)
ax2.legend(fontsize=10, loc='best')
ax2.grid(False)
ax2.tick_params(axis='both', labelsize=10)

fig.patch.set_facecolor('white')
for ax in [ax1, ax2]:
    ax.set_facecolor('white')
    ax.spines['bottom'].set_color('black')
    ax.spines['top'].set_color('black')
    ax.spines['left'].set_color('black')
    ax.spines['right'].set_color('black')

plt.tight_layout()
plt.show()

In [ ]:
import pyvista as pv
import numpy as np

mesh = pv.read(r"C:\Users\KU5001153\Downloads\bp_50percent_147.vtk")
x_positions = np.linspace(100, 500, 15)
plotter = pv.Plotter(shape=(2, 2))

# Setup subplots
plotter.subplot(0, 0)
plotter.add_text("Velocity Magnitude", font_size=10)
plotter.subplot(0, 1)
plotter.add_text("du/dx", font_size=10)
plotter.subplot(1, 0)
plotter.add_text("dv/dx", font_size=10)
plotter.subplot(1, 1)
plotter.add_text("dw/dx", font_size=10)

spacing = 20  # Space between slices

# First do velocity magnitude separately
plotter.subplot(0, 0)
for i, x_pos in enumerate(x_positions):
    slice_x = mesh.slice(normal='x', origin=(x_pos, 0, 0))
    clipped_slice = slice_x.clip(normal=[-1, 0, 0], origin=[0, 0, 0])
    clipped_slice = clipped_slice.clip(normal=[0, 0, -1], origin=[0, 0, 0])
    clipped_slice = clipped_slice.clip(normal=[0, -1, 0], origin=[0, 70, 0])
    clipped_slice.translate((0, i*spacing, 0))
    u = clipped_slice.get_array('U')
    velocity_magnitude = np.linalg.norm(u, axis=1)
    clipped_slice['velocity_magnitude'] = velocity_magnitude
    plotter.add_mesh(clipped_slice,
                    scalars='velocity_magnitude',
                    opacity=0.8,
                    cmap='rainbow',
                    clim=[0, 10],
                    scalar_bar_args={'title': 'Velocity Magnitude (m/s)'})

# Then do gradients in other subplots
for i, x_pos in enumerate(x_positions):
    if i > 0:
        # Current slice
        slice_x = mesh.slice(normal='x', origin=(x_pos, 0, 0))
        clipped_slice = slice_x.clip(normal=[-1, 0, 0], origin=[0, 0, 0])
        clipped_slice = clipped_slice.clip(normal=[0, 0, -1], origin=[0, 0, 0])
        clipped_slice = clipped_slice.clip(normal=[0, -1, 0], origin=[0, 70, 0])
        clipped_slice.translate((0, i*spacing, 0))
        u = clipped_slice.get_array('U')
        
        # Previous slice with same clipping
        prev_slice = mesh.slice(normal='x', origin=(x_positions[i-1], 0, 0))
        prev_slice = prev_slice.clip(normal=[-1, 0, 0], origin=[0, 0, 0])
        prev_slice = prev_slice.clip(normal=[0, 0, -1], origin=[0, 0, 0])
        prev_slice = prev_slice.clip(normal=[0, -1, 0], origin=[0, 70, 0])
        prev_u = prev_slice.get_array('U')
        
        # Calculate gradients
        dx = x_pos - x_positions[i-1]
        du_dx = (u[:, 0] - prev_u[:, 0]) / dx
        dv_dx = (u[:, 1] - prev_u[:, 1]) / dx
        dw_dx = (u[:, 2] - prev_u[:, 2]) / dx
        
        clipped_slice['du_dx'] = du_dx
        clipped_slice['dv_dx'] = dv_dx
        clipped_slice['dw_dx'] = dw_dx
        
        # Add to gradient subplots
        plotter.subplot(0, 1)
        plotter.add_mesh(clipped_slice,
                        scalars='du_dx',
                        opacity=0.8,
                        cmap='rainbow',
                        clim=[-0.1, 0.1],
                        scalar_bar_args={'title': 'du/dx (1/s)'})
        
        plotter.subplot(1, 0)
        plotter.add_mesh(clipped_slice,
                        scalars='dv_dx',
                        opacity=0.8,
                        cmap='rainbow',
                        clim=[-0.1, 0.1],
                        scalar_bar_args={'title': 'dv/dx (1/s)'})
        
        plotter.subplot(1, 1)
        plotter.add_mesh(clipped_slice,
                        scalars='dw_dx',
                        opacity=0.8,
                        cmap='rainbow',
                        clim=[-0.1, 0.1],
                        scalar_bar_args={'title': 'dw/dx (1/s)'})

# Add axes to all subplots
for i in range(2):
    for j in range(2):
        plotter.subplot(i, j)
        plotter.add_axes()

plotter.window_size = [1600, 1200]
plotter.camera.zoom(1.1)
plotter.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

def create_rotor_grid(R, nr=50, npsi=72):
    """Create a grid of points across the rotor disk"""
    x = np.linspace(-R, R, nr)
    y = np.linspace(-R, R, nr)
    X, Y = np.meshgrid(x, y)
    R_grid = np.sqrt(X**2 + Y**2)
    PSI_grid = np.arctan2(Y, X)
    mask = R_grid <= R
    return X, Y, R_grid, PSI_grid, mask

def velocity_field(x, y, R, omega, V_inf):
    """Calculate velocity components for streamlines"""
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    
    # Rotational velocity components
    v_rot_x = -omega * y
    v_rot_y = omega * x
    
    # Forward flight components
    v_fwd_x = V_inf * np.ones_like(x)
    v_fwd_y = np.zeros_like(y)
    
    # Combined velocities
    v_x = v_rot_x + v_fwd_x
    v_y = v_rot_y + v_fwd_y
    
    # Mask points outside disk
    mask = r <= R
    v_x[~mask] = 0
    v_y[~mask] = 0
    
    return v_x, v_y, r <= R

# Parameters
R = 0.12  # radius in meters
omega = 565  # rad/s
V_inf = 10  # m/s forward velocity

# Create grid
x, y, r_grid, psi_grid, mask = create_rotor_grid(R)

# Calculate velocity field
v_x, v_y, valid_mask = velocity_field(x, y, R, omega, V_inf)

# Create normalized velocity magnitude for coloring
v_mag = np.sqrt(v_x**2 + v_y**2)
v_mag[~valid_mask] = np.nan

# Create figure
fig = plt.figure(figsize=(15, 12))

# 1. Streamlines with velocity magnitude
ax1 = fig.add_subplot(221)
contour = ax1.contourf(x, y, v_mag/(omega*R), levels=20, cmap='inferno')
# Add streamlines with varying color and density
ax1.streamplot(x, y, v_x, v_y, color='white', density=2, linewidth=0.5,
              arrowsize=1, arrowstyle='->')
fig.colorbar(contour, ax=ax1, label='Normalized Velocity Magnitude')
ax1.set_title('Velocity Magnitude and Streamlines')
ax1.set_aspect('equal')

# 2. Advancing/Retreating side visualization
ax2 = fig.add_subplot(222)
advance_ratio = V_inf * np.cos(psi_grid) / (omega * R)
advance_ratio[~mask] = np.nan
contour2 = ax2.contourf(x, y, advance_ratio, levels=20, cmap='inferno')
ax2.streamplot(x, y, v_x, v_y, color='white', density=2, linewidth=0.5)
fig.colorbar(contour2, ax=ax2, label='Advance Ratio')
ax2.set_title('Advancing/Retreating Sides')
ax2.set_aspect('equal')

# 3. Polar view with streamlines
ax3 = fig.add_subplot(223, projection='polar')
r = np.linspace(0, R, 50)
theta = np.linspace(0, 2*np.pi, 72)
r_grid_polar, theta_grid = np.meshgrid(r, theta)
v_total = np.sqrt((omega * r_grid_polar)**2 + V_inf**2 + 
                  2*omega*r_grid_polar*V_inf*np.cos(theta_grid))
polar_contour = ax3.contourf(theta_grid, r_grid_polar, v_total/(omega*R), 
                            levels=20, cmap='viridis')
# Add blade positions for reference
ax3.plot([0, 0], [0, R], 'r-', linewidth=2, label='Blades')
ax3.plot([np.pi, np.pi], [0, R], 'r-', linewidth=2)
# Add forward flight direction indicator
ax3.plot([0], [R*1.1], 'b>', markersize=10, label='V∞')
fig.colorbar(polar_contour, ax=ax3, label='Normalized Velocity')
ax3.set_title('Polar View')
ax3.legend()

# 4. Velocity profiles at different radial positions
ax4 = fig.add_subplot(224)
r_cuts = [0.3, 0.6, 0.9]  # r/R values
theta = np.linspace(0, 2*np.pi, 100)
for r_cut in r_cuts:
    r = r_cut * R
    v_rot = omega * r
    v_total = np.sqrt(v_rot**2 + V_inf**2 + 2*v_rot*V_inf*np.cos(theta))
    ax4.plot(np.degrees(theta), v_total/(omega*R), 
             label=f'r/R = {r_cut:.1f}')
ax4.grid(True)
ax4.set_xlabel('Azimuth Angle (degrees)')
ax4.set_ylabel('Normalized Total Velocity')
ax4.set_title('Velocity Variation with Azimuth')
ax4.legend()

plt.tight_layout()
plt.show()

# Print key parameters
print("\nFlow Parameters:")
print(f"Tip Speed (ΩR): {omega * R:.2f} m/s")
print(f"Forward Speed (V∞): {V_inf:.2f} m/s")
print(f"Advance Ratio (μ): {V_inf/(omega*R):.3f}")
print(f"Max Combined Velocity: {np.nanmax(v_mag):.2f} m/s")

In [ ]:
import numpy as np
import pyvista as pv

# Set white background theme
pv.set_plot_theme('document')

def create_rotor_grid(R, nr=200):
    """Create a grid of points across the rotor disk"""
    x = np.linspace(-R, R, nr)
    y = np.linspace(-R, R, nr)
    X, Y = np.meshgrid(x, y)
    Z = np.zeros_like(X)
    R_grid = np.sqrt(X**2 + Y**2)
    PSI_grid = np.arctan2(Y, X)
    mask = R_grid <= R
    return X, Y, Z, R_grid, PSI_grid, mask

def velocity_field(x, y, R, omega, V_inf):
    """Calculate velocity components"""
    r = np.sqrt(x**2 + y**2)
    
    # Rotational velocity components
    v_rot_x = -omega * y
    v_rot_y = omega * x
    
    # Forward flight components
    v_fwd_x = V_inf * np.ones_like(x)
    v_fwd_y = np.zeros_like(y)
    
    # Combined velocities
    v_x = v_rot_x + v_fwd_x
    v_y = v_rot_y + v_fwd_y
    v_z = np.zeros_like(x)
    
    # Mask points outside disk
    mask = r <= R
    v_x[~mask] = 0
    v_y[~mask] = 0
    
    return v_x, v_y, v_z, mask

def calculate_thrust(r_grid, psi_grid, omega, R, V_inf, rho=1.225):
    """Calculate thrust distribution"""
    V_rot = omega * r_grid
    V_forward = V_inf * np.cos(psi_grid)
    V_total = np.sqrt(V_rot**2 + V_forward**2)
    
    r_norm = r_grid/R
    tip_loss = 0.5 * (1 + np.tanh(20*(1 - r_norm)))
    root_cutout = (1 - np.exp(-5*r_norm))
    
    dT = 0.5 * rho * V_total**2 * r_norm * tip_loss * root_cutout
    dT = dT * (1 + 0.2 * np.cos(psi_grid))
    
    return dT

# Parameters
R = 0.12  # radius in meters
omega = 565  # rad/s
V_inf = 10  # m/s forward velocity

# Create grid
x, y, z, r_grid, psi_grid, mask = create_rotor_grid(R)

# Calculate fields
v_x, v_y, v_z, valid_mask = velocity_field(x, y, R, omega, V_inf)
v_mag = np.sqrt(v_x**2 + v_y**2)
thrust = calculate_thrust(r_grid, psi_grid, omega, R, V_inf)

# Create PyVista grid
grid = pv.StructuredGrid(x, y, z)

# Add data to grid
grid.point_data["velocity"] = v_mag.flatten()
grid.point_data["thrust"] = thrust.flatten()
grid.point_data["vectors"] = np.column_stack((v_x.flatten(), v_y.flatten(), v_z.flatten()))

# Create plotter with white background
pl = pv.Plotter(shape=(1, 2), window_size=[1600, 800], background='white')

# Velocity plot
pl.subplot(0, 0)
pl.add_mesh(grid, scalars="velocity", cmap="viridis", 
           show_edges=False, clim=[0, np.max(v_mag)])

# Add streamlines using seed points
n_seeds = 20
theta = np.linspace(0, 2*np.pi, n_seeds)
seed_points = np.column_stack((
    0.8*R*np.cos(theta),
    0.8*R*np.sin(theta),
    np.zeros_like(theta)
))
seed_points = pv.PolyData(seed_points)

# Generate streamlines
streamlines = grid.streamlines_from_source(
    seed_points,
    vectors="vectors",
    max_time=100,
    initial_step_length=0.01,
    integration_direction='both'
)
pl.add_mesh(streamlines, color='black', line_width=2)

# Add rotor disk outline
circle = pv.Circle(radius=R)
pl.add_mesh(circle, color='gray', style='wireframe', line_width=2)

# Add blades
blade1 = pv.Line((0,0,0), (R,0,0))
blade2 = pv.Line((0,0,0), (-R,0,0))
pl.add_mesh(blade1, color='red', line_width=3)
pl.add_mesh(blade2, color='red', line_width=3)

pl.add_title("Velocity Field", color='black')
pl.view_xy()
pl.camera.zoom(0.8)  # Zoomed out more

# Thrust plot
pl.subplot(0, 1)
pl.add_mesh(grid, scalars="thrust", cmap="inferno", 
           show_edges=False, clim=[0, np.max(thrust)])

# Add rotor disk outline
pl.add_mesh(circle, color='gray', style='wireframe', line_width=2)
pl.add_mesh(blade1, color='red', line_width=3)
pl.add_mesh(blade2, color='red', line_width=3)

pl.add_title("Thrust Distribution", color='black')
pl.view_xy()
pl.camera.zoom(0.8)  # Zoomed out more

# Add text with black color
text = f"V∞ = {V_inf} m/s, Ω = {omega} rad/s, R = {R} m"
pl.add_text(text, position='upper_left', color='black', font_size=12)

# Show the plot
pl.show()

# Print parameters
print("\nRotor Parameters:")
print(f"Tip Speed (ΩR): {omega * R:.2f} m/s")
print(f"Forward Speed (V∞): {V_inf:.2f} m/s")
print(f"Advance Ratio (μ): {V_inf/(omega*R):.3f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

def create_rotor_grid(R, nr=200):
    """Create a grid of points across the rotor disk"""
    x = np.linspace(-R, R, nr)
    y = np.linspace(-R, R, nr)
    X, Y = np.meshgrid(x, y)
    R_grid = np.sqrt(X**2 + Y**2)
    PSI_grid = np.arctan2(Y, X)
    mask = R_grid <= R
    return X, Y, R_grid, PSI_grid, mask

def velocity_field(x, y, R, omega, V_inf):
    """Calculate velocity components"""
    r = np.sqrt(x**2 + y**2)
    
    # Rotational velocity components
    v_rot_x = -omega * y
    v_rot_y = omega * x
    
    # Forward flight components
    v_fwd_x = V_inf * np.ones_like(x)
    v_fwd_y = np.zeros_like(y)
    
    # Combined velocities
    v_x = v_rot_x + v_fwd_x
    v_y = v_rot_y + v_fwd_y
    
    # Mask points outside disk
    mask = r <= R
    v_x[~mask] = np.nan  # Changed from 0 to nan
    v_y[~mask] = np.nan  # Changed from 0 to nan
    
    return v_x, v_y, mask

# Parameters
R = 0.12  # radius in meters
omega = 565  # rad/s
V_inf = 10  # m/s forward velocity

# Create grid
x, y, r_grid, psi_grid, mask = create_rotor_grid(R)

# Calculate fields
v_x, v_y, valid_mask = velocity_field(x, y, R, omega, V_inf)
v_mag = np.sqrt(v_x**2 + v_y**2)

# Calculate thrust with smoother transitions
r_norm = r_grid/R
tip_loss = 0.5 * (1 + np.tanh(20*(1 - r_norm)))
root_cutout = (1 - np.exp(-5*r_norm))
V_total = np.sqrt((omega * r_grid)**2 + V_inf**2 + 2*omega*r_grid*V_inf*np.cos(psi_grid))
thrust = 0.5 * 1.225 * V_total**2 * r_norm * tip_loss * root_cutout
thrust = thrust * (1 + 0.2 * np.cos(psi_grid))
thrust[~mask] = np.nan

# Create figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# 1. Velocity Field
contour1 = ax1.contourf(x/R, y/R, v_mag, levels=50, cmap='viridis', extend='both')
# Streamlines with more starting points
y_start = np.linspace(-0.8, 0.8, 20)
x_start = np.array([-0.8] * len(y_start))
ax1.streamplot(x/R, y/R, v_x, v_y, 
              start_points=np.column_stack((x_start, y_start)),
              density=2, color='black', linewidth=1)

# 2. Thrust Distribution
contour2 = ax2.contourf(x/R, y/R, thrust, levels=50, cmap='inferno', extend='both')

# Common settings for both plots
for ax, title in zip([ax1, ax2], ['Velocity Field', 'Thrust Distribution']):
    # Add rotor disk edge
    circle = plt.Circle((0, 0), 1, fill=False, color='gray', linestyle='--', linewidth=1)
    ax.add_artist(circle)
    
    # Add blades
    ax.plot([0, 1], [0, 0], 'r-', linewidth=2)
    ax.plot([0, -1], [0, 0], 'r-', linewidth=2)
    
    # Add forward flight direction
    ax.arrow(-1.2, 0, 0.2, 0, head_width=0.05, head_length=0.05, 
             fc='blue', ec='blue')
    
    ax.set_aspect('equal')
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.grid(True, linestyle=':', alpha=0.3)
    ax.set_xlabel('x/R')
    ax.set_ylabel('y/R')
    ax.set_title(title, pad=20, fontsize=14)

# Add colorbars with formatted labels
cbar1 = fig.colorbar(contour1, ax=ax1)
cbar1.set_label('Velocity (m/s)', fontsize=12)
cbar2 = fig.colorbar(contour2, ax=ax2)
cbar2.set_label('Thrust (N)', fontsize=12)

# Add parameters text
plt.figtext(0.5, 0.02, 
           f'V∞ = {V_inf} m/s    Ω = {omega} rad/s    R = {R} m', 
           ha='center', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pyvista as pv
from scipy.interpolate import griddata
# Read the mesh

x_positions = np.linspace(100, 500, 400)
y_positions = np.linspace(70, 130, 60) # Adjusted y range around y=100
z_height = 50  # Fixed z-height

# Initialize arrays to store velocity components
u_surface = np.zeros((len(y_positions), len(x_positions)))
v_surface = np.zeros((len(y_positions), len(x_positions)))
w_surface = np.zeros((len(y_positions), len(x_positions)))


mesh = pv.read(r"C:\Users\KU5001153\Downloads\bp_50percent_75.vtk")

# Extract values at each position
for i, x_pos in enumerate(x_positions):
    for j, y_pos in enumerate(y_positions):
        # Create and clip slice
        slice_x = mesh.slice(normal='x', origin=(x_pos, 0, 0))
        clipped_slice = slice_x.clip(normal=[-1, 0, 0], origin=[0, 0, 0])
        clipped_slice = clipped_slice.clip(normal=[0, 0, -1], origin=[0, 0, 0])
        
        # Get points and velocities
        points = clipped_slice.points
        u = clipped_slice.get_array('U')
        
        # Find point closest to desired y and z position
        distances = np.sqrt((points[:, 1] - y_pos)**2 + (points[:, 2] - z_height)**2)
        closest_point_idx = np.argmin(distances)
        
        # Store velocity components
        u_surface[j, i] = u[closest_point_idx, 0]
        v_surface[j, i] = u[closest_point_idx, 1]
        w_surface[j, i] = u[closest_point_idx, 2]

# Create finer mesh for interpolation
x_fine = np.linspace(x_positions.min(), x_positions.max(), 200)
y_fine = np.linspace(y_positions.min(), y_positions.max(), 200)
X_fine, Y_fine = np.meshgrid(x_fine, y_fine)

# Create the points array from original x,y positions instead
X_grid, Y_grid = np.meshgrid(x_positions, y_positions)
points = np.column_stack((X_grid.ravel(), Y_grid.ravel()))

# Now interpolate
U_smooth = griddata(points, u_surface.ravel(), (X_fine, Y_fine), method='cubic')

# Create separate figures for each component
fig_u = plt.figure(figsize=(12, 8), dpi=100, facecolor='white')
fig_v = plt.figure(figsize=(12, 8), dpi=100, facecolor='white')
fig_w = plt.figure(figsize=(12, 8), dpi=100, facecolor='white')

def setup_plot(ax, title):
    ax.zaxis.set_tick_params(pad=10)
    ax.set_xlabel('X (m)', labelpad=10, fontname='Arial')
    ax.set_ylabel('Y (m)', labelpad=10, fontname='Arial')
    ax.set_zlabel('Velocity (m/s)', labelpad=10, fontname='Arial')
    ax.tick_params(axis='both', which='major', labelsize=9)
    for label in ax.get_xticklabels() + ax.get_yticklabels() + ax.get_zticklabels():
        label.set_fontname('Arial')
    ax.grid(False)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.xaxis.pane.set_edgecolor('none')
    ax.yaxis.pane.set_edgecolor('none')
    ax.zaxis.pane.set_edgecolor('none')
    ax.view_init(elev=20, azim=140)
    ax.zaxis.set_major_formatter(plt.FormatStrFormatter('%.2f'))
    ax.set_box_aspect([1, 1, 0.6], zoom=0.8)
    ax.set_title(title, fontname='Arial', pad=20)

# Interpolate and plot U component
ax_u = fig_u.add_subplot(111, projection='3d', computed_zorder=False)
U_smooth = griddata(points, u_surface.flatten(), (X_fine, Y_fine), method='cubic')
surf_u = ax_u.plot_surface(X_fine, Y_fine, U_smooth,
                          cmap='inferno',
                          antialiased=True)
setup_plot(ax_u, 'U-velocity Component')
ax_u.set_zlim(U_smooth.min(), U_smooth.max())
ax_u.set_zticks(np.linspace(U_smooth.min(), U_smooth.max(), 6))
fig_u.colorbar(surf_u, ax=ax_u, pad=0.1, aspect=20)

# Interpolate and plot V component
ax_v = fig_v.add_subplot(111, projection='3d', computed_zorder=False)
V_smooth = griddata(points, v_surface.flatten(), (X_fine, Y_fine), method='cubic')
surf_v = ax_v.plot_surface(X_fine, Y_fine, V_smooth,
                          cmap='inferno',
                          antialiased=True)
setup_plot(ax_v, 'V-velocity Component')
ax_v.set_zlim(V_smooth.min(), V_smooth.max())
ax_v.set_zticks(np.linspace(V_smooth.min(), V_smooth.max(), 6))
fig_v.colorbar(surf_v, ax=ax_v, pad=0.1, aspect=20)

ax_w = fig_w.add_subplot(111, projection='3d', computed_zorder=False)
W_smooth = griddata(points, w_surface.flatten(), (X_fine, Y_fine), method='cubic')
surf_w = ax_w.plot_surface(X_fine, Y_fine, W_smooth,
                          cmap='inferno',
                          antialiased=True)
setup_plot(ax_w, 'W-velocity Component')
ax_w.set_zlim(W_smooth.min(), W_smooth.max())
ax_w.set_zticks(np.linspace(W_smooth.min(), W_smooth.max(), 6))
fig_w.colorbar(surf_w, ax=ax_w, pad=0.1, aspect=20)

plt.show()